In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.argv = [""]

In [ ]:
from cartpole.init import initialize
from cartpole.trainer import PPOTrainer

In [4]:
params = initialize(seconds_since_epoch=1766250823)
trainer = PPOTrainer(params)
trainer()

/home/mspcvsp/miniconda/envs/cage2_env/lib/python3.10/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


global_step=512, avg_episodic_reward=22.61, avg_episodic_length=23

iteration: 1 avg_clipfracs: 0.0261 avg_old_approx_kl: 0.000333 avg_approx_kl: 0.002889
abg_pg_loss: -0.0073 avg_v_loss: 49.0479 avg_entropy_loss: 0.6902 avg_loss: 24.5097

global_step=1024, avg_episodic_reward=24.54, avg_episodic_length=25

iteration: 2 avg_clipfracs: 0.0408 avg_old_approx_kl: 0.005460 avg_approx_kl: 0.003414
abg_pg_loss: -0.0066 avg_v_loss: 36.2556 avg_entropy_loss: 0.6774 avg_loss: 18.1144

global_step=1536, avg_episodic_reward=29.40, avg_episodic_length=29

iteration: 3 avg_clipfracs: 0.0142 avg_old_approx_kl: 0.002211 avg_approx_kl: 0.001590
abg_pg_loss: -0.0030 avg_v_loss: 54.1963 avg_entropy_loss: 0.6594 avg_loss: 27.0885

global_step=2048, avg_episodic_reward=36.19, avg_episodic_length=36

iteration: 4 avg_clipfracs: 0.0323 avg_old_approx_kl: 0.002880 avg_approx_kl: 0.003056
abg_pg_loss: -0.0047 avg_v_loss: 52.6326 avg_entropy_loss: 0.6416 avg_loss: 26.3052

global_step=2560, avg_episodic_reward

KeyboardInterrupt: 